# Global mean differences and GWLs

This notebook calculates global mean differences of SST and STAS with respect to global mean temperature, and GWL for each of the 26 global climate model

Loading libraires:

In [30]:
import netCDF4 as ncÇ
import numpy as np
import numpy.ma as ma
from scipy import signal
import warnings
import os

Ignore warnings:

In [31]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter("ignore", category=RuntimeWarning)

Define functions:

In [32]:
def robust(mean_hist, mean_future, avg_period):
    mean_hist_detrend = signal.detrend(np.isnan(mean_hist))
    treshold = np.sqrt(2)*1.645*mean_hist_detrend.std()/np.sqrt(avg_period)
    if abs(mean_future) > treshold:
        data_robust = 1
    else:
        data_robust = 0
    return data_robust

# Read strings from a txt file
def read_txt(fname, ncolumns, dtype):
    fdata   = open(fname)
    flines  = np.array(fdata.readlines())
    data_in = np.empty([len(flines)])
    if dtype == 'string':
        data_in = np.empty([len(flines)], dtype=object)
    for i in np.arange(len(flines)):
        data_in[i]=flines[i].split()[ncolumns-1]      
    return(data_in)
    fdata.close()
    
# Read a netcdf file
def read_nc(fname, varname, ifmerged):
    if ifmerged:
        f  = nc.MFDataset(fname)
    else:
        f  = nc.Dataset(fname)
    var    = f.variables[varname]
    lon    = f.variables['lon']
    lat    = f.variables['lat']
    time   = f.variables['time']
    dates  = nc.num2date(time[:], time.units, time.calendar)
    return var, lon, lat, time, dates

Define paths and enviromental variables:

In [33]:
# Define paths
resolution   = 'low_res'
seasons      = ['ANNUAL', 'DJF', 'MAM', 'JJA', 'SON'] 
varname      = 'tos'    # 'tas' 'stas'
avg_period   = 10       # 10, 20, 30
ssps         = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
gwls         = [1.5, 2, 3, 4]
ref_ntimes   = 51 
ystart_to_plot  = 2010
member_names = f'models_{resolution}.txt'
root = '..'
path2info  = f'{root}/data/data_info/' # path to files provifing info on e.g. model names or regins

Read list of the models to be processed:

In [34]:
member_list = f'{path2info}/{member_names}'
members  = read_txt(member_list, 1, 'string')
nmembers = len(members)

Main code:

In [ ]:
# Loop over seasons
for season in seasons: 
    print("Working on the season: " + season)
    path2data   = f'{root}/data/data_raw/{resolution}/{varname}/'
    path2out    = f'{root}/data/data_txt/{resolution}/global/{varname}_test/'

    if not os.path.exists(path2out):
        os.makedirs(path2out) 
    
    # Loop over ssp scenarion
    for ssp in ssps:
        
        # Load files
        print("  Working on the ssp: " + ssp)
        fname = [f'{path2data}/CMIP6_historical_{varname}_{season}.nc',
                 f'{path2data}/CMIP6_{ssp}_{varname}_{season}.nc']
        
        var, lon, lat, time, dates_all = read_nc(fname, varname, True)
        
        # Load projection times
        years_all  = [date.year for date in dates_all]
        ntimes_all = len(years_all)

        # Define reference period - from 1850-1900
        dates_ref  = dates_all[0:ref_ntimes]
        ntimes_ref = len(dates_ref)

        # Define analysys period
        dates  = dates_all[ref_ntimes:]
        ntimes = len(dates)

        # Read years and months for the period of anlysys
        dyears = [date.year for date in dates]

        # Define start and year for tickmarks
        year_start = int(dyears[0] + avg_period / 2 )
        year_end = int(dyears[ntimes-1] - avg_period / 2 + 1 )
        years = np.arange(year_start, year_end, 10)

        # Calculate historical mean
        print("    Calculating the historical mean ...")
        ref_mean = np.nanmean(var[0:ref_ntimes, :, :, :], axis=(0)) # not weighted
        mask = ma.masked_invalid(ref_mean[:]).mask

        # Calculate weights
        xx, yy  = np.meshgrid(lon, lat)
        weights = np.cos(np.deg2rad(yy))
        weights_masked = ma.masked_array(weights, mask[0,:,:])

        # Calculate running mean
        print("    Calculating the running mean ...")
        run_ntimes = int(ntimes_all - avg_period - ref_ntimes + 1)        
        diff_mean = np.empty([run_ntimes , nmembers])

        for t in np.arange(0, run_ntimes):
            it = t + ref_ntimes
            diff_2D_mean       = np.nanmean(var[it:(it + avg_period-1), :, :, :], axis=(0)) - ref_mean
            diff_2D_weghted    = diff_2D_mean*weights_masked/np.nanmean(weights_masked)
            diff_mean[t, :] = np.round(np.nanmean(diff_2D_weghted, axis=(1,2)),3)
            

        # Check the robustenss of the signal
        ifrobust = np.empty([run_ntimes])
        robustness = np.empty([run_ntimes, nmembers])

        for i in range(run_ntimes):
            plus  = 100.*float(np.count_nonzero(diff_mean[i,:] >= 0))/float(nmembers)
            minus = 100.*float(np.count_nonzero(diff_mean[i,:] < 0))/float(nmembers)

            for j in range(nmembers):
                robustness[i,j]=robust(ref_mean[:,j], diff_mean[i,j], avg_period)
                if (100.*float(np.count_nonzero(robustness[i,:] == 1.))/float(nmembers)) >= 66. and max(plus,minus) >= 80.:
                    ifrobust[i]=1.
                elif (100.*float(np.count_nonzero(robustness[i,:] == 1.))/float(nmembers)) >= 66. and max(plus,minus) < 80.:
                    ifrobust[i]=2.
                else:
                    ifrobust[i]=0.


        # Define when a gwl reached
        indx_gwls = np.empty([nmembers, len(gwls)])
        year_gwls = np.empty([nmembers, len(gwls)])
        for i in np.arange(0, nmembers):
            for j in np.arange(0, len(gwls)):
                indx_gwls[i,j] = np.argmax(diff_mean[:, i] > gwls[j])
                if indx_gwls[i,j] == 0:
                    year_gwls[i,j] = np.nan
                else:
                    year_gwls[i,j] = float(dyears[int(indx_gwls[i,j])]+int(avg_period/2-1))

                    f'{path2out}/Beta_{varname}_{season}.csv'

        # Save running mean, gwls, and info on robustens in txt files
        output = f'{varname}_{ssp}_{season}' 
        np.savetxt(f'{path2out}/{output}.csv', diff_mean, fmt='%10.3f')
        np.savetxt(f'{path2out}/{output}_gwls.csv', year_gwls, fmt='%10.0f')
        np.savetxt(f'{path2out}/ifrobust_{output}.csv', ifrobust, fmt='%10.0f')